<a href="https://colab.research.google.com/github/albertotrunk/depth2video/blob/main/03_img2img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy

In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline

# load the pipeline
model_id = "stabilityai/stable-diffusion-2-1"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
).to("cuda")



In [ ]:
import requests
from PIL import Image
from IPython.display import display
import numpy as np
from torchvision.transforms import GaussianBlur

url_mask = "https://raw.githubusercontent.com/albertotrunk/depth2video/main/example/depth.png?raw=true" #@param {type:"string"}
mask_image = Image.open(requests.get(url_mask, stream=True).raw).convert('RGB')
display(mask_image)

url_init = "https://raw.githubusercontent.com/albertotrunk/depth2video/main/example/color.png?raw=true" #@param {type:"string"}
init_image = Image.open(requests.get(url_init, stream=True).raw).convert('RGB')
display(init_image)


src = np.array(init_image)
mask = np.array(mask_image)

print(mask.dtype, mask.min(), mask.max())
# uint8 0 255

mask = mask / 255

print(mask.dtype, mask.min(), mask.max())
# float64 0.0 1.0

dst = src * mask

masked_img = Image.fromarray(dst.astype(np.uint8))
masked_img

In [ ]:

#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
prompt = "a photo of Scarlett Johansson with a short haircut wearing a white sweater, hyper resolution 10k" #@param {type:"string"}
negative_prompt = "border, frame, lowres, jpeg artifacts, low quality, normal quality, worst quality, poorly drawn, error. abstract, asymmetrical, blurry, cropped, disconnected, duplicate, extra, missing, signature, text, username, watermark. amputee, bad anatomy, deformed, disfigured, disproportionate, eye bags, fused, malformed, morbid, mutated, mutation, mutilated, hands, fingers,bad hand,extra limbs,missing limb,extra leg,missing leg,missing hand,extra hand"  #@param {type:"string"}


images = pipe(prompt=prompt, image=masked_img, strength=0.41, guidance_scale=9.5, negative_prompt=negative_prompt).images

image = images[0]
image

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = [init_image,mask_image,masked_img,image]


plt.figure(figsize=(20,10))
columns = 4
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(image)